In [1]:
import numpy as np
from scipy.linalg import lu_solve, lu_factor

In [2]:
# Just some matrixes that I have been using as examples,
# and guidance to implement the solution.

test_A = [[14, 14, -9, 3, -5], 
            [14, 52, -15, 2, -32], 
            [-9, -15, 36, -5, 16], 
            [3, 2, -5, 47, 49], 
            [-5, -32, 16, 49, 79]]

test_B = [-15, -100,106,329,463]

# matrixA = [[9.0, 3.0, 2.0, 0.0, 7.0], 
#             [7.0, 6.0, 9.0, 6.0, 4.0], 
#             [2.0, 7.0, 7.0, 8.0, 2.0], 
#             [0.0, 9.0, 7.0, 2.0, 2.0], 
#             [7.0, 3.0, 6.0, 4.0, 3.0]]

# matrixB = [35.0, 58.0, 53.0, 37.0, 39.0]

In [3]:
def infty_norm(matrix):
    row_sum = np.array([])

    for i in range(0, matrix.shape[0]):
        row = np.sum(np.abs(matrix[i]))
        row_sum = np.append(row_sum, row)

    return row_sum.max()

In [4]:
def v(matrix, vector, iterations):
    for i in range(0, iterations+1):
        vector = vector / infty_norm(vector)
        vector = lu_solution(matrix, vector)
    
    return infty_norm(vector)

In [5]:
def k(matrix, vector, iterations):
    return v(matrix, vector, iterations) * infty_norm(matrix)

In [6]:
def lu_solution(matrix, vector):
    
    # This will make a copy of the original matrix and column-vector
    # which will be used to find the solution x for the system Ax = b.
    matrix_A = matrix.copy()
    matrix_b = vector.copy()
    
    # Making sure that float numbers will be used.
    matrix_A = np.array(matrix_A, dtype=np.float) #float or float32, or float64 ?
    matrix_b = np.array(matrix_b, dtype=np.float)

    indx = np.arange(0, matrix_A.shape[0])

    for i in range(0, matrix_A.shape[0]-1):
        am = np.abs(matrix_A[i, i])
        p = i


        for j in range(i+1, matrix_A.shape[0]):
            if np.abs(matrix_A[j, i]) > am:
                am = np.abs(matrix_A[j, i])
                p = j

        if p > i:        
            for k in range(0, matrix_A.shape[0]):
                hold = matrix_A[i,k]
                matrix_A[i, k] = matrix_A[p, k]
                matrix_A[p, k] = hold

            ihold = indx[i]
            indx[i] = indx[p]
            indx[p] = ihold


        for j in range(i+1, matrix_A.shape[0]):
            matrix_A[j, i] = matrix_A[j, i] / matrix_A[i, i]

            for k in range(i+1, matrix_A.shape[0]):
                matrix_A[j, k] = matrix_A[j, k] - matrix_A[j, i] * matrix_A[i, k]

    # matrix_A
    # matrix_b
    # indx

    x = np.zeros(matrix_A.shape[0])

    for k in range(0, matrix_A.shape[0]):
        x[k] = matrix_b[indx[k]]

    for k in range(0, matrix_A.shape[0]):
        matrix_b[k] = x[k]

    # x
    # matrix_b

    y = np.zeros(matrix_A.shape[0])
    y[0] = matrix_b[0]

    for i in range(1, matrix_A.shape[0]):
        sum = 0.0

        for j in range(0, i):
            sum = sum + matrix_A[i, j] * y[j]

        y[i] = (matrix_b[i] - sum)

    # y

    x[-1] = y[-1] / matrix_A[-1, -1]

    for i in range(matrix_A.shape[0]-1, -1, -1):
        sum = 0.0

        for j in range(i+1, matrix_A.shape[0]):
            sum = sum + matrix_A[i, j] * x[j]

        x[i] = (y[i] - sum) / matrix_A[i, i]

    return x

In [7]:
lu_solution(test_A, test_B)

array([1.73829205e-14, 1.00000000e+00, 2.00000000e+00, 3.00000000e+00,
       4.00000000e+00])

In [8]:
matrixA = np.array([[1, 1/2, 1/3], 
               [1/2, 1/3, 1/4], 
               [1/3, 1/4, 1/5]], 
              dtype=np.float)

y0 = np.array([0.2190, 
               0.0470, 
               0.6789], 
              dtype = np.float)

In [9]:
# Note that k* is defined as alpha * v. Where alpha is the
# infinity-norm of the matrix A.

infty_norm(matrixA) * v(matrixA, y0, 4)

682.2111537287485

In [10]:
# Python equivalent to MATLAB cond function.

np.linalg.cond(matrixA)

524.0567775860627

In [11]:
# The rcond function from MATLAB is the reciprocal
# of the cond function.

1 / np.linalg.cond(matrixA)

0.0019081901861974792